In [1]:
import pandas as pd

In [2]:
# Read in the CSVs needed to make the join.
ticker = pd.read_csv('data/asx-tickers.csv')
headlines = pd.read_csv('data/abcnews-date-text.csv')

In [3]:
ticker.head(2)

,ticker,company,industry
0,1AD,ADALTA LIMITED,"Pharmaceuticals, Biotechnology & Life Sciences"
1,1AG,ALTERRA LIMITED,Commercial & Professional Services


In [4]:
headlines.head(2)

,publish_date,headline_text
0,20030219,aba decides against community broadcasting lic...
1,20030219,act fire witnesses must be aware of defamation


In [5]:
import numpy as np
from multiprocessing import cpu_count, Pool
from tqdm import tqdm

cores = cpu_count()
 
def parallelize(data, func):
    data_split = np.array_split(data, cores)
    pool = Pool(cores)
    data = pd.concat(tqdm(pool.map(func, data_split)))
    pool.close()
    pool.join()
    return data

In [ ]:
from fuzzywuzzy import process, fuzz
tqdm.pandas(desc="HALP")

# Fuzzy match function
def fuzzy_match(x, choices, scorer, cutoff):
    return process.extractOne(
        x, choices=choices, scorer=scorer, score_cutoff=cutoff
    )

# Parallelization function, to run on each split of data
def appfun(df):
    return df.loc[:, 'headline_text'].progress_apply(
        fuzzy_match,
        args=(
            ticker.loc[:, 'company'], 
            fuzz.token_set_ratio,
            80
        )
    )

# Run in parallel
matching_results = parallelize(headlines, appfun)

HALP:   1%|          | 840/137958 [00:47<2:01:07, 18.87it/s] 